In [2]:
import pandas as pd
import json
import string
import ijson
import csv

** Pull in our csv files, and build the feature vectors from them **

In [3]:
df_review = pd.read_csv('dataset/review_with_text.csv')
df_review = df_review.drop('Unnamed: 0',axis=1)
df_review.head()

/home/ec2-user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,review_id,user_id,business_id,stars,text
0,v0i_UHJMo_hPBq9bxWvW4w,bv2nCi5Qv5vroFiqKGopiw,0W4lkclzZThpx3V65bVgig,5.0,"Love the staff, love the meat, love the place...."
1,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5.0,Super simple place but amazing nonetheless. It...
2,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5.0,Small unassuming place that changes their menu...
3,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5.0,Lester's is located in a beautiful neighborhoo...
4,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4.0,Love coming here. Yes the place always needs t...


In [4]:
df_user = pd.read_csv('dataset/user.csv')
df_user = df_user.drop('Unnamed: 0',axis=1)
df_user = df_user.set_index(df_user['user_id'])
df_user = df_user.drop('user_id',axis=1)
df_user.head()

,average_stars,review_count,useful
user_id,,,
oMy_rEb0UBEmMlu-zcxnoQ,4.67,8,0
JJ-aSuM4pCFPdkfoZ34q0Q,3.70,10,0
uUzsFQn_6cXDh6rPNGbIFA,2.00,1,0
mBneaEEH5EMyxaVyqS-72A,4.67,6,0
W5mJGs-dcDWRGEhAzUYtoA,4.67,3,0


In [5]:
df_business = pd.read_csv('dataset/business.csv')
df_business = df_business.drop('Unnamed: 0',axis=1)
df_business = df_business.set_index(df_business['business_id'])
df_business = df_business.drop('business_id',axis=1)
df_business.head()

,stars,review_count
business_id,,
FYWN1wneV18bWNgQjJ2GNg,4.0,22
He-G7vWjzVUysIKrfNbPUQ,3.0,11
KQPW8lFf1y5BT2MxiSZ3QA,1.5,18
8DShNS-LuFqpEWIp0HxijA,3.0,9
PfOCPjBrlQAnz__NXj9h_w,3.5,116


In [6]:
columns = ['review_id','user_stars','user_review_count','user_useful','business_stars','business_review_count','unigram','class']

featureVecs = pd.DataFrame(columns=columns)
featureVecs.head()

,review_id,user_stars,user_review_count,user_useful,business_stars,business_review_count,unigram,class


In [11]:
fp = open("unigramPredictions.json")
unigramdict = json.load(fp)

len(unigramdict)

50001

In [12]:
fp = open("dataset/unigramPredictions50000.json")
unigramdict = json.load(fp)

review = pd.DataFrame(columns=columns)
reviews = []

i = 0
for index,row in df_review.iterrows():
    review.at[0,'review_id'] = row['review_id']
    review.at[0,'class'] = row['stars']
    
    review.at[0,'user_stars'] = df_user['average_stars'][row['user_id']]
    review.at[0,'user_review_count'] = df_user['review_count'][row['user_id']]
    review.at[0,'user_useful'] = df_user['useful'][row['user_id']]
    
    review.at[0,'business_stars'] = df_business['stars'][row['business_id']]
    review.at[0,'business_review_count'] = df_business['review_count'][row['business_id']]
    
    review.at[0,'unigram'] = unigramdict[row['review_id']]
    
    reviews.append(review)
    review = pd.DataFrame(columns=columns)
    
    i += 1
    if i%5000 == 0:
        print(i)
    
    if i == 50000:
        break



5000
10000
15000
20000
25000
30000
35000
40000
45000
50000


In [9]:
featureVecs = pd.concat(reviews)
featureVecs = featureVecs.set_index(featureVecs['review_id'])
featureVecs = featureVecs.drop('review_id',axis=1)
featureVecs.to_csv("./dataset/feature_vectors.csv")
featureVecs.head()

,user_stars,user_review_count,user_useful,business_stars,business_review_count,unigram,class
review_id,,,,,,,
v0i_UHJMo_hPBq9bxWvW4w,4.67,6,0,4,1953,5,5
vkVSCC7xljjrAI4UGfnKEQ,4.67,6,0,4,84,5,5
n6QzIUObkYshz4dz2QRJTw,4.67,6,0,4.5,50,5,5
MV3CcKScW05u5LVfF6ok0g,4.67,6,0,4,70,5,5
IXvOzsEMYtiJI0CARmj77Q,4.67,6,0,3.5,61,4,4
